In [1]:
!pip install kaggle
!pip install accelerate peft bitsandbytes pip install git+https://github.com/huggingface/transformers trl py7zr auto-gptq optimum

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-v5w81pho
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-v5w81pho
  Resolved https://github.com/huggingface/transformers to commit 0864dd3beb238b7bec3528a3d1d6c17a28f51a51
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 113.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 

# Download Dataset

In [2]:
!mkdir ~/.kaggle

In [3]:
!cp kaggle.json ~/.kaggle/

In [4]:
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download gabrieltardochi/wikipedia-movie-plots-with-plot-summaries/

 99% 31.0M/31.2M [00:02<00:00, 18.6MB/s]
100% 31.2M/31.2M [00:02<00:00, 15.0MB/s]


In [6]:
!unzip /content/wikipedia-movie-plots-with-plot-summaries.zip

Archive:  /content/wikipedia-movie-plots-with-plot-summaries.zip
  inflating: wiki_movie_plots_deduped_with_summaries.csv  


In [7]:
from huggingface_hub import notebook_login
notebook_login()

In [8]:
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM, prepare_model_for_kbit_training, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig, TrainingArguments
from trl import SFTTrainer
import os
from sklearn.model_selection import train_test_split
import pandas as pd

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


# Dataset Reading, Cleaning and Preprocessing

In [9]:
data = pd.read_csv('/content/wiki_movie_plots_deduped_with_summaries.csv')
data.shape

(34886, 9)

In [10]:
data.columns

Index(['Release Year', 'Title', 'Origin/Ethnicity', 'Director', 'Cast',
       'Genre', 'Wiki Page', 'Plot', 'PlotSummary'],
      dtype='object')

In [11]:
data.head()

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,PlotSummary
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr...",Carrie Nation and her followers burst into a s...
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov...","The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed...","The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...,The first shot is set in a wood during winter ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...,The earliest known adaptation of the classic f...


In [12]:
selected_columns = ['Plot', 'PlotSummary']

# Display the first few rows of the DataFrame with selected columns
data[selected_columns].head()

,Plot,PlotSummary
0,"A bartender is working at a saloon, serving dr...",Carrie Nation and her followers burst into a s...
1,"The moon, painted with a smiling face hangs ov...","The moon, painted with a smiling face hangs ov..."
2,"The film, just over a minute long, is composed...","The film, just over a minute long, is composed..."
3,Lasting just 61 seconds and consisting of two ...,The first shot is set in a wood during winter ...
4,The earliest known adaptation of the classic f...,The earliest known adaptation of the classic f...


In [13]:
data['Plot'][100]

"Grace Roberts (played by Lea Leland), marries rancher Edward Smith, who is revealed to be a neglectful, vice-ridden spouse. They have a daughter, Vivian. Dr. Franklin (Leonid Samoloff) whisks Grace away from this unhappy life, and they move to New York under aliases, pretending to be married (since surely Smith would not agree to a divorce). Grace and Franklin have a son, Walter (Milton S. Gould). Vivian gets sick, however, and Grace and Franklin return to save her. Somehow this reunion, as Smith had assumed Grace to be dead, causes the death of Franklin. This plot device frees Grace to return to her father's farm with both children.[1]"

In [14]:
data['PlotSummary'][100]

'Grace Roberts marries rancher Edward Smith, who is revealed to be a neglectful, vice-ridden spouse. Dr. Franklin (Leonid Samoloff) whisks Grace away from this unhappy life, and they move to New York under aliases, pretending to be married. Grace and Franklin have a son, Walter (Milton S. Gould)'

In [15]:
data["text"] = data[selected_columns].apply(lambda x: "###Human: Summarize this following dialogue: " + x["Plot"] + "\n###Assistant: " +x["PlotSummary"], axis=1)

In [16]:
data.columns

Index(['Release Year', 'Title', 'Origin/Ethnicity', 'Director', 'Cast',
       'Genre', 'Wiki Page', 'Plot', 'PlotSummary', 'text'],
      dtype='object')

In [17]:
df = data.drop(columns=['Release Year', 'Title', 'Origin/Ethnicity', 'Director', 'Cast', 'Genre', 'Wiki Page'])

In [18]:
df.shape

(34886, 3)

In [19]:
df.head()

,Plot,PlotSummary,text
0,"A bartender is working at a saloon, serving dr...",Carrie Nation and her followers burst into a s...,###Human: Summarize this following dialogue: A...
1,"The moon, painted with a smiling face hangs ov...","The moon, painted with a smiling face hangs ov...",###Human: Summarize this following dialogue: T...
2,"The film, just over a minute long, is composed...","The film, just over a minute long, is composed...",###Human: Summarize this following dialogue: T...
3,Lasting just 61 seconds and consisting of two ...,The first shot is set in a wood during winter ...,###Human: Summarize this following dialogue: L...
4,The earliest known adaptation of the classic f...,The earliest known adaptation of the classic f...,###Human: Summarize this following dialogue: T...


In [20]:
df = Dataset.from_pandas(df)

In [21]:
df

Dataset({
    features: ['Plot', 'PlotSummary', 'text'],
    num_rows: 34886
})

# Model and Tokenizer

In [22]:
tokenizer = AutoTokenizer.from_pretrained("TheBloke/Mistral-7B-Instruct-v0.1-GPTQ")
tokenizer.pad_token = tokenizer.eos_token
quantization_config_loading = GPTQConfig(bits=4, disable_exllama=True, tokenizer=tokenizer)
model = AutoModelForCausalLM.from_pretrained(
                          "TheBloke/Mistral-7B-Instruct-v0.1-GPTQ",
                          quantization_config=quantization_config_loading,
                          device_map="auto"
                    )
print(model)
model.config.use_cache=False
model.config.pretraining_tp=1
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    r=16, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM", target_modules=["q_proj", "v_proj"]
)
model = get_peft_model(model, peft_config)

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.


config.json:   0%|          | 0.00/963 [00:00<?, ?B/s]

You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute and has already quantized weights. However, loading attributes (e.g. use_exllama, exllama_config, use_cuda_fp16, max_input_length) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.


model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (rotary_emb): MistralRotaryEmbedding()
          (k_proj): QuantLinear()
          (o_proj): QuantLinear()
          (q_proj): QuantLinear()
          (v_proj): QuantLinear()
        )
        (mlp): MistralMLP(
          (act_fn): SiLU()
          (down_proj): QuantLinear()
          (gate_proj): QuantLinear()
          (up_proj): QuantLinear()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm()
  )
  (lm_head): Linear(in_features=4096, out_features=32000, bias=False)
)


In [27]:
training_arguments = TrainingArguments(
        output_dir="mistral-finetuned-wikipedia-movie-AI-plot-summary",
        per_device_train_batch_size=8,
        gradient_accumulation_steps=1,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=100,
        num_train_epochs=4,
        max_steps=250,
        fp16=True,
        push_to_hub=True
    )

In [24]:
trainer = SFTTrainer(
        model=model,
        train_dataset=df,
        peft_config=peft_config,
        dataset_text_field="text",
        args=training_arguments,
        tokenizer=tokenizer,
        packing=False,
        max_seq_length=512
    )

Map:   0%|          | 0/34886 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:247: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [ ]:
trainer.train()

In [30]:
trainer.push_to_hub()

'https://huggingface.co/ErnestBeckham/mistral-finetuned-wikipedia-movie-AI-plot-summary/tree/main/'

In [31]:
from peft import AutoPeftModelForCausalLM
from transformers import GenerationConfig
from transformers import AutoTokenizer
import torch
tokenizer = AutoTokenizer.from_pretrained("ErnestBeckham/mistral-finetuned-wikipedia-movie-AI-plot-summary")

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

In [35]:
inputs = tokenizer("""
###Human: Summarize this following dialogue: WiFi hacking, also known as WiFi cracking, is the practice of unauthorized access to a wireless network. With the increasing use of wireless networks in homes, businesses, and public spaces, WiFi hacking has become a significant concern. Hackers use various techniques to gain unauthorized access to WiFi networks, which can lead to theft of sensitive information, unauthorized usage of bandwidth, and other malicious activities. WiFi networks are vulnerable to hacking because they transmit data through radio waves, which can be intercepted by anyone within range. Hackers use specialized software and hardware to intercept and decode these radio waves, allowing them to gain access to the network. Once they have access, they can use various techniques to maintain their connection, even after the legitimate user has logged off. There are several types of WiFi hacking, including password cracking, rogue access point (AP) attacks, and evil twin attacks. Password cracking involves attempting to guess or brute-force the password of a WiFi network. Rogue AP attacks involve setting up an unauthorized access point that mimics a legitimate network, in order to trick devices into connecting to it. Evil twin attacks involve creating a fake WiFi network that looks identical to a legitimate one, in order to trick devices into connecting to it and capturing sensitive information. WiFi hacking can have serious consequences, both for individuals and organizations. Hackers can use WiFi networks to steal sensitive information, such as login credentials, financial data, and personal data. They can also use WiFi networks to launch other types of attacks, such as denial-of-service (DoS) attacks, malware attacks, and ransomware attacks. In addition, WiFi hacking can lead to unauthorized usage of bandwidth, which can slow down the network and cause inconvenience to legitimate users. To protect against WiFi hacking, it is essential to implement strong security measures. This includes using strong passwords, encrypting data, using firewalls and antivirus software, and regularly updating software and firmware. Additionally, organizations should implement policies and procedures for WiFi security, such as limiting access to sensitive information and implementing two-factor authentication. WiFi hacking is a serious concern that can have significant consequences for individuals and organizations. It is important to understand the different types of WiFi hacking and the techniques used by hackers to gain unauthorized access to wireless networks. By implementing strong security measures and staying vigilant, individuals and organizations can protect their WiFi networks from hacking and ensure the security of their sensitive information.
###Assistant: """, return_tensors="pt").to("cuda")

model = AutoPeftModelForCausalLM.from_pretrained(
    "/content/mistral-finetuned-wikipedia-movie-AI-plot-summary",
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="cuda")

generation_config = GenerationConfig(
    do_sample=True,
    top_k=1,
    temperature=0.1,
    max_new_tokens=25,
    pad_token_id=tokenizer.eos_token_id
)

In [36]:
import time
st_time = time.time()
outputs = model.generate(**inputs, generation_config=generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(time.time()-st_time)


###Human: Summarize this following dialogue: WiFi hacking, also known as WiFi cracking, is the practice of unauthorized access to a wireless network. With the increasing use of wireless networks in homes, businesses, and public spaces, WiFi hacking has become a significant concern. Hackers use various techniques to gain unauthorized access to WiFi networks, which can lead to theft of sensitive information, unauthorized usage of bandwidth, and other malicious activities. WiFi networks are vulnerable to hacking because they transmit data through radio waves, which can be intercepted by anyone within range. Hackers use specialized software and hardware to intercept and decode these radio waves, allowing them to gain access to the network. Once they have access, they can use various techniques to maintain their connection, even after the legitimate user has logged off. There are several types of WiFi hacking, including password cracking, rogue access point (AP) attacks, and evil twin atta